# 🎯 Identificación de Parámetros - Principio de Mínima Acción

## Formulación Lagrangiana del Levitador Magnético

**Lagrangiano:** $\mathcal{L} = \frac{1}{2}m\dot{y}^2 + mgy + \frac{1}{2}L(y)i^2$

**Parámetros a identificar (5D):** $k_0, k, a, m, R$

**Función de pérdida Physics-Informed:**
$$\mathcal{J} = W_{data} \cdot MSE_{traj} + W_{EL} \cdot \|R_{EL}\|^2 + W_{E} \cdot \|R_{energy}\|^2$$
---

In [ ]:
# Setup
import subprocess, sys, platform
print("🔍 Verificando GPU...")
try:
    print(subprocess.check_output(['nvidia-smi'], text=True))
except: print("⚠️ Sin GPU")

IN_COLAB = 'google.colab' in sys.modules
!pip install -q "numpy<2.1.0" "ml-dtypes>=0.4.0,<0.5.0"
if IN_COLAB:
    !pip install -q "jax[cuda12]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
else:
    !pip install -q jax jaxlib
!pip install -q scipy pandas matplotlib tqdm
print("✅ Dependencias instaladas")

In [ ]:
import jax
import jax.numpy as jnp
from jax import random, jit, vmap
from jax.lax import scan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json, time, os
from datetime import datetime
from tqdm.auto import tqdm
from functools import partial

print(f"JAX devices: {jax.devices()}")
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

In [ ]:
REPO_URL = "https://github.com/JRavenelco/levitador-benchmark.git"
if not os.path.exists("levitador-benchmark"):
    !git clone {REPO_URL}
    %cd levitador-benchmark
elif os.getcwd().split(os.sep)[-1] != "levitador-benchmark":
    %cd levitador-benchmark
!ls -la data/*.txt 2>/dev/null || echo "No data"
!ls -la data/sesiones_kan_pinn/*.txt 2>/dev/null || echo "No KAN-PINN data" 

In [ ]:
def load_data(filepath, subsample=1):
    df = pd.read_csv(filepath, sep=r'\s+', comment='#', header=None)
    if subsample > 1: df = df.iloc[::subsample].reset_index(drop=True)
    return {
        't': df.iloc[:,0].values.astype(np.float32),
        'y': df.iloc[:,1].values.astype(np.float32),
        'i': df.iloc[:,2].values.astype(np.float32),
        'u': df.iloc[:,3].values.astype(np.float32)
    }

DATA_FILE = "data/datos_levitador.txt"
SUBSAMPLE = 10
data = load_data(DATA_FILE, SUBSAMPLE)
print(f"✅ Datos: {len(data['t'])} muestras, {data['t'][0]:.2f}-{data['t'][-1]:.2f}s")

fig, ax = plt.subplots(3,1,figsize=(12,6))
ax[0].plot(data['t'], data['y']*1000); ax[0].set_ylabel('y [mm]')
ax[1].plot(data['t'], data['i']); ax[1].set_ylabel('i [A]')
ax[2].plot(data['t'], data['u']); ax[2].set_ylabel('u [V]'); ax[2].set_xlabel('t [s]')
plt.tight_layout(); plt.show()

In [ ]:
t_jax = jnp.array(data['t'])
y_jax = jnp.array(data['y'])
i_jax = jnp.array(data['i'])
u_jax = jnp.array(data['u'])

dt_jax = jnp.diff(t_jax, prepend=t_jax[0])
dt_jax = dt_jax.at[0].set(dt_jax[1])
dt_mean = float(jnp.mean(dt_jax))

dy_jax = jnp.gradient(y_jax, dt_mean)
ddy_jax = jnp.gradient(dy_jax, dt_mean)
di_jax = jnp.gradient(i_jax, dt_mean)

y0, i0 = float(y_jax[0]), float(i_jax[0])
print(f"✅ GPU: dt={dt_mean:.4f}s, y0={y0*1000:.2f}mm, i0={i0:.3f}A")

## Modelo Físico Lagrangiano
$$L(y) = k_0 + \frac{k}{1+y/a}, \quad \frac{dL}{dy} = -\frac{k}{a(1+y/a)^2}$$
**Residuos E-L:** $R_{mec} = m\ddot{y} - \frac{1}{2}i^2\frac{dL}{dy} - mg$, $R_{elec} = L\dot{i} + \frac{dL}{dy}\dot{y}i + Ri - u$

In [ ]:
GRAVITY = 9.81
M_REF, R_REF, K0_REF, A_REF = 0.009, 2.2, 0.0035, 0.0052
print(f"Ref: m={M_REF*1000}g, R={R_REF}Ω, k0={K0_REF*1000}mH, a={A_REF*1000}mm")

@jit
def simulate_trajectory(params, u_data, dt_data, y0, i0, g):
    k0, k, a, m, R = params
    state = jnp.array([y0, 0.0, i0])
    def step(state, inputs):
        u, dt = inputs
        y, dy, i = state
        denom = 1.0 + y/a
        L = k0 + k/denom
        dL = -k/(a * denom**2)
        ddy = (0.5*i*i*dL + m*g)/m
        di = (u - R*i - dL*dy*i)/L
        y_new = jnp.clip(y + dy*dt, 0, 0.03)
        i_new = jnp.clip(i + di*dt, 0, 5)
        return jnp.array([y_new, dy+ddy*dt, i_new]), jnp.array([y_new, i_new, dy+ddy*dt])
    _, out = scan(step, state, (u_data, dt_data))
    return out[:,0], out[:,1], out[:,2]

@jit 
def euler_lagrange_residuals(y, dy, ddy, i, di, u, params):
    k0, k, a, m, R = params
    denom = 1.0 + y/a
    L = k0 + k/denom
    dL = -k/(a * denom**2)
    R_mec = m*ddy - 0.5*i**2*dL - m*GRAVITY
    R_elec = L*di + dL*dy*i + R*i - u
    return R_mec, R_elec

euler_lagrange_vec = jit(vmap(euler_lagrange_residuals, in_axes=(0,0,0,0,0,0,None)))
print("✅ Modelo Lagrangiano definido")

In [ ]:
W_POS, W_CUR, W_EL, W_EN = 10000.0, 1.0, 100.0, 10.0
print(f"Pesos: W_pos={W_POS}, W_cur={W_CUR}, W_EL={W_EL}, W_energy={W_EN}")

@jit
def fitness_physics(params, u_data, dt_data, y_data, i_data, dy_data, ddy_data, di_data, y0, i0, g, dt):
    y_sim, i_sim, dy_sim = simulate_trajectory(params, u_data, dt_data, y0, i0, g)
    mse_y = jnp.mean((y_data - y_sim)**2)
    mse_i = jnp.mean((i_data - i_sim)**2)
    
    ddy_sim = jnp.gradient(dy_sim, dt)
    di_sim = jnp.gradient(i_sim, dt)
    R_mec, R_elec = euler_lagrange_vec(y_sim, dy_sim, ddy_sim, i_sim, di_sim, u_data, params)
    mse_el = jnp.mean(R_mec**2)/(params[3]**2) + jnp.mean(R_elec**2)
    
    k0, k, a, m, R = params
    L_ind = k0 + k/(1.0 + y_sim/a)
    E = 0.5*m*dy_sim**2 - m*g*y_sim + 0.5*L_ind*i_sim**2
    dE = jnp.gradient(E, dt)
    R_en = dE - (u_data*i_sim - R*i_sim**2)
    mse_en = jnp.mean(R_en**2)
    
    return jnp.nan_to_num(W_POS*mse_y + W_CUR*mse_i + W_EL*mse_el + W_EN*mse_en, nan=1e6)

fitness_pop = jit(vmap(fitness_physics, in_axes=(0,None,None,None,None,None,None,None,None,None,None,None)))
print("✅ Fitness Physics-Informed definido")

In [ ]:
def de_lagrangian(fitness_fn, bounds, pop_size=200, F=0.7, CR=0.85, max_iter=400, seed=42, ref=None):
    key = random.PRNGKey(seed)
    n = bounds.shape[0]
    ranges = bounds[:,1] - bounds[:,0]
    
    key, sk = random.split(key)
    pop = random.uniform(sk, (pop_size, n)) * ranges + bounds[:,0]
    if ref is not None:
        key, sk = random.split(key)
        seeded = ref + random.normal(sk, (pop_size//4, n))*0.1*ranges
        pop = pop.at[:pop_size//4].set(jnp.clip(seeded, bounds[:,0], bounds[:,1]))
    
    fit = fitness_fn(pop)
    best_idx = jnp.argmin(fit)
    best, best_fit = pop[best_idx], fit[best_idx]
    hist = {'best': [float(best_fit)], 'mean': [float(jnp.mean(fit))], 'params': [best.tolist()]}
    
    @jit
    def step(pop, best, key):
        k1, k2, k3 = random.split(key, 3)
        a, b = random.randint(k1, (pop_size,), 0, pop_size), random.randint(k2, (pop_size,), 0, pop_size)
        mut = best + F*(pop[a] - pop[b])
        trial = jnp.where(random.uniform(k3, (pop_size, n)) < CR, mut, pop)
        return jnp.clip(trial, bounds[:,0], bounds[:,1])
    
    for gen in tqdm(range(max_iter), desc="DE"):
        key, sk = random.split(key)
        trial = step(pop, best, sk)
        trial_fit = fitness_fn(trial)
        improved = trial_fit < fit
        pop = jnp.where(improved[:,None], trial, pop)
        fit = jnp.where(improved, trial_fit, fit)
        idx = jnp.argmin(fit)
        if fit[idx] < best_fit: best, best_fit = pop[idx], fit[idx]
        hist['best'].append(float(best_fit))
        hist['mean'].append(float(jnp.mean(fit)))
        if gen % 50 == 0: hist['params'].append(best.tolist())
    
    hist['params'].append(best.tolist())
    print(f"\n✅ Óptimo: k0={best[0]*1000:.3f}mH, k={best[1]*1000:.3f}mH, a={best[2]*1000:.3f}mm, m={best[3]*1000:.2f}g, R={best[4]:.3f}Ω")
    return best, best_fit, hist

print("✅ DE Lagrangiano definido")

In [ ]:
BOUNDS = jnp.array([
    [0.001, 0.020],  # k0 [H]
    [0.001, 0.020],  # k [H]
    [0.002, 0.015],  # a [m]
    [0.005, 0.020],  # m [kg]
    [1.5, 4.0]       # R [Ω]
])
REF = jnp.array([K0_REF, K0_REF, A_REF, M_REF, R_REF])

print("Límites:", [(f"{b[0]*1000:.1f}-{b[1]*1000:.1f}mH" if i<2 else f"{b[0]*1000:.1f}-{b[1]*1000:.1f}mm" if i==2 else f"{b[0]*1000:.0f}-{b[1]*1000:.0f}g" if i==3 else f"{b[0]:.1f}-{b[1]:.1f}Ω") for i,b in enumerate(BOUNDS)])

fit_fn = partial(fitness_pop, u_data=u_jax, dt_data=dt_jax, y_data=y_jax, i_data=i_jax, 
                 dy_data=dy_jax, ddy_data=ddy_jax, di_data=di_jax, y0=y0, i0=i0, g=GRAVITY, dt=dt_mean)

t0 = time.time()
best_params, best_fit, history = de_lagrangian(fit_fn, BOUNDS, pop_size=200, max_iter=400, ref=REF)
print(f"⏱️ Tiempo: {time.time()-t0:.1f}s")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Convergence
axes[0,0].semilogy(history['best'], 'b-', lw=2, label='Best')
axes[0,0].semilogy(history['mean'], 'r--', alpha=0.5, label='Mean')
axes[0,0].set_xlabel('Gen'); axes[0,0].set_ylabel('Fitness'); axes[0,0].legend(); axes[0,0].grid(True)
axes[0,0].set_title('Convergencia', fontweight='bold')

# Inductance params evolution
ph = np.array(history['params'])
gens = np.linspace(0, len(history['best'])-1, len(ph))
axes[0,1].plot(gens, ph[:,0]*1000, 'b-o', ms=4, label='k0 [mH]')
axes[0,1].plot(gens, ph[:,1]*1000, 'r-s', ms=4, label='k [mH]')
axes[0,1].plot(gens, ph[:,2]*1000, 'g-^', ms=4, label='a [mm]')
axes[0,1].axhline(K0_REF*1000, c='b', ls=':', alpha=0.5)
axes[0,1].axhline(A_REF*1000, c='g', ls=':', alpha=0.5)
axes[0,1].set_xlabel('Gen'); axes[0,1].legend(); axes[0,1].grid(True)
axes[0,1].set_title('Parámetros Inductancia', fontweight='bold')

# Position comparison
y_opt, i_opt, _ = simulate_trajectory(best_params, u_jax, dt_jax, y0, i0, GRAVITY)
axes[1,0].plot(data['t'], data['y']*1000, 'b-', lw=1.5, label='Exp', alpha=0.7)
axes[1,0].plot(data['t'], np.array(y_opt)*1000, 'r--', lw=1.5, label='Sim')
axes[1,0].set_xlabel('t [s]'); axes[1,0].set_ylabel('y [mm]'); axes[1,0].legend(); axes[1,0].grid(True)
axes[1,0].set_title('Posición', fontweight='bold')

# Current comparison
axes[1,1].plot(data['t'], data['i'], 'b-', lw=1.5, label='Exp', alpha=0.7)
axes[1,1].plot(data['t'], np.array(i_opt), 'r--', lw=1.5, label='Sim')
axes[1,1].set_xlabel('t [s]'); axes[1,1].set_ylabel('i [A]'); axes[1,1].legend(); axes[1,1].grid(True)
axes[1,1].set_title('Corriente', fontweight='bold')

plt.tight_layout(); plt.savefig('convergence.png', dpi=150); plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(15, 5))

ph = np.array(history['params'])
k0h, kh, ah = ph[:,0]*1000, ph[:,1]*1000, ph[:,2]*1000
colors = np.linspace(0, 1, len(k0h))

# 3D trajectory
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(k0h, kh, ah, c=colors, cmap='viridis', s=50)
ax1.plot(k0h, kh, ah, 'k-', alpha=0.3)
ax1.scatter([k0h[0]], [kh[0]], [ah[0]], c='g', s=200, marker='o', label='Inicio')
ax1.scatter([k0h[-1]], [kh[-1]], [ah[-1]], c='r', s=200, marker='*', label='Óptimo')
ax1.scatter([K0_REF*1000], [K0_REF*1000], [A_REF*1000], c='blue', s=150, marker='D', label='Ref')
ax1.set_xlabel('k0 [mH]'); ax1.set_ylabel('k [mH]'); ax1.set_zlabel('a [mm]')
ax1.set_title('Evolución 3D (k0, k, a)', fontweight='bold'); ax1.legend(fontsize=8)

# k0 vs k
ax2 = fig.add_subplot(132)
ax2.scatter(k0h, kh, c=colors, cmap='viridis', s=30)
ax2.plot(k0h, kh, 'k-', alpha=0.3)
ax2.scatter(k0h[0], kh[0], c='g', s=150, marker='o', label='Inicio')
ax2.scatter(k0h[-1], kh[-1], c='r', s=150, marker='*', label='Óptimo')
ax2.scatter(K0_REF*1000, K0_REF*1000, c='blue', s=100, marker='D', label='Ref')
ax2.set_xlabel('k0 [mH]'); ax2.set_ylabel('k [mH]'); ax2.legend(); ax2.grid(True)
ax2.set_title('k0 vs k', fontweight='bold')

# k0 vs a
ax3 = fig.add_subplot(133)
ax3.scatter(k0h, ah, c=colors, cmap='viridis', s=30)
ax3.plot(k0h, ah, 'k-', alpha=0.3)
ax3.scatter(k0h[0], ah[0], c='g', s=150, marker='o', label='Inicio')
ax3.scatter(k0h[-1], ah[-1], c='r', s=150, marker='*', label='Óptimo')
ax3.scatter(K0_REF*1000, A_REF*1000, c='blue', s=100, marker='D', label='Ref')
ax3.set_xlabel('k0 [mH]'); ax3.set_ylabel('a [mm]'); ax3.legend(); ax3.grid(True)
ax3.set_title('k0 vs a', fontweight='bold')

plt.tight_layout(); plt.savefig('inductance_3d.png', dpi=150); plt.show()
print("✅ Visualización 3D guardada")

In [ ]:
print("="*60)
print("📊 RESULTADOS FINALES - PRINCIPIO DE MÍNIMA ACCIÓN")
print("="*60)
print(f"\n🎯 Parámetros Identificados:")
print(f"   k0 = {float(best_params[0])*1000:.4f} mH  (Inductancia base)")
print(f"   k  = {float(best_params[1])*1000:.4f} mH  (Coef. inductancia)")
print(f"   a  = {float(best_params[2])*1000:.4f} mm  (Parámetro geométrico)")
print(f"   m  = {float(best_params[3])*1000:.4f} g   (Masa)")
print(f"   R  = {float(best_params[4]):.4f} Ω    (Resistencia)")

print(f"\n📐 Comparación con Referencia (Santana 2023):")
print(f"   k0: {float(best_params[0])*1000:.4f} vs {K0_REF*1000:.4f} mH (Δ={abs(float(best_params[0])-K0_REF)*1000:.4f})")
print(f"   a:  {float(best_params[2])*1000:.4f} vs {A_REF*1000:.4f} mm (Δ={abs(float(best_params[2])-A_REF)*1000:.4f})")
print(f"   m:  {float(best_params[3])*1000:.4f} vs {M_REF*1000:.4f} g  (Δ={abs(float(best_params[3])-M_REF)*1000:.4f})")
print(f"   R:  {float(best_params[4]):.4f} vs {R_REF:.4f} Ω  (Δ={abs(float(best_params[4])-R_REF):.4f})")

print(f"\n✅ Fitness final: {float(best_fit):.6e}")

# Save results
results = {
    'params': {'k0': float(best_params[0]), 'k': float(best_params[1]), 'a': float(best_params[2]), 
               'm': float(best_params[3]), 'R': float(best_params[4])},
    'fitness': float(best_fit),
    'reference': {'k0': K0_REF, 'a': A_REF, 'm': M_REF, 'R': R_REF},
    'timestamp': datetime.now().isoformat()
}
with open('lagrangian_results.json', 'w') as f:
    json.dump(results, f, indent=2)
print(f"\n💾 Resultados guardados en 'lagrangian_results.json'")